In [2]:
# import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader
from torchvision import transforms, models, datasets
import torch
from torch import nn
from torch.autograd import Variable
# from tqdm import tqdm_notebook
import torch.optim as optim
import pickle
from torch.utils.data import Dataset
import os
from torchvision.datasets import ImageFolder
from torchvision.datasets.folder import IMG_EXTENSIONS
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torch.utils.data import DataLoader
import tqdm
from tqdm import tqdm_notebook
import cv2


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(2362944, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 4)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

    


net = Net()

In [5]:
class LoadData(Dataset):
    def __init__(self, dir_path, label, transforms=None):
        train_ls = os.listdir(dir_path)
        train_ls.sort()
        self.train = {i : (dir_path + x) for i,x in enumerate(train_ls)}
        self.target=torch.zeros(len(train_ls))+label
        self.transforms = transforms
        
    def __len__(self):
        return len(self.train)

    def __getitem__(self, idx):
        img = Image.open(self.train[idx])
        img = cv2.resize(img, (400,400), interpolation = cv2.INTER_AREA)
        img.load()
        img = np.asarray(img )
        if self.transforms:
            img = self.transforms(img)
            label=self.target[idx].long()
            
        return img, label

In [6]:
transform = transforms.Compose([transforms.ToTensor()])
isbi = LoadData('datum/l11/', 0, transforms=transform)

In [8]:
print isbi.target
print len(isbi)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
39


In [9]:
epochs=100
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [10]:
loader = DataLoader(isbi, batch_size=1, shuffle=True)
for epoch in range(epochs):  # loop over the dataset multiple times
    t = tqdm_notebook(enumerate(loader), total=len(loader))
    running_loss = 0.0
    for i, (inputs,labels) in t:
#         print "hello"
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 3 == 2:    # print every 3 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 3))
            running_loss = 0.0
print('Finished Training')

NameError: global name 'cv2' is not defined

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images,labels= data.train, torch.max(data.target.data,1)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += 1
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))
